In [8]:
# 必要なライブラリをインポート
!pip install selenium

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import datetime

def extract_stock_data(driver):
    WebDriverWait(driver, 10).until(
        lambda d: d.find_element(By.CSS_SELECTOR, "div.highcharts-tooltip table").is_displayed()
    )

    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html, "html.parser")
    tooltip = soup.select_one("div.highcharts-tooltip table")

    if not tooltip:
        return None

    tds = tooltip.select("td")
    if len(tds) < 5:
        return None

    # 日付の整形
    try:
        y, m, d = map(int, tds[0].text.strip().split('/'))
        date = f"{y:04d}-{m:02d}-{d:02d}"
    except:
        return None

    # 株価4項目
    prices = [td.text.strip().replace(',', '') for td in tds[1:5]]
    return [date] + prices


def get_stock_values(driver, url):
    driver.get(url)
    time.sleep(5)

    chart = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'g.highcharts-series-group'))
    )
    actions = ActionChains(driver)
    # グラフ中央にマウスを移動
    chart_size = chart.size
    width = chart_size['width']
    height = chart_size['height']
    center_x = width // 2
    center_y = height // 2
    start_x = center_x + width // 2  # 右端から開始
    print(f"チャート幅: {width}px")

    #actions.move_to_element_with_offset(chart, start_x, center_y).perform()
    actions.move_to_element(chart).perform()
    actions.move_by_offset(width // 2, 0).perform()
    time.sleep(0.5)

    data_points = []
    seen_dates = set()

    for x_offset in range(width, 0, -1):  # 右端から左端へ1pxずつ移動
        try:
            # actions.move_to_element_with_offset(chart, x_offset, 0).perform()
            actions.move_by_offset(-1, 0).perform()
            time.sleep(0.03)

            stock_data = extract_stock_data(driver)
            print(stock_data)
            if stock_data and stock_data[0] not in seen_dates:
                data_points.append(stock_data)
                seen_dates.add(stock_data[0])

        except Exception as e:
            print(f"移動エラー: {e}")
            continue

    return data_points


def main():
    start_time = time.time()

    # ヘッドレスモードで起動するためのオプションを設定
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument("--window-size=1920,1080") #ブラウザウィンドウサイズを広げる

    # Chromeを立ち上げる
    chrome_driver = webdriver.Chrome(options=chrome_options)

    try:
        url = 'https://www.nikkei.com/markets/worldidx/chart/nk225/?type=6month'
        stock_data_list = get_stock_values(chrome_driver, url)

        end_time = time.time()
        duration = end_time - start_time
        print(f"スクレイピング時間: {duration:.2f}秒\n")

        for data in stock_data_list:
            print(", ".join(data))
    finally:
        chrome_driver.quit()

if __name__ == "__main__":
    main()



チャート幅: 559px
['2025-08-15', '始値: 42866.38', '高値: 43405.43', '安値: 42736.86', '終値: 43378.31']
['2025-08-15', '始値: 42866.38', '高値: 43405.43', '安値: 42736.86', '終値: 43378.31']
['2025-08-15', '始値: 42866.38', '高値: 43405.43', '安値: 42736.86', '終値: 43378.31']
['2025-08-15', '始値: 42866.38', '高値: 43405.43', '安値: 42736.86', '終値: 43378.31']
['2025-08-14', '始値: 43129.18', '高値: 43199.83', '安値: 42606.73', '終値: 42649.26']
['2025-08-14', '始値: 43129.18', '高値: 43199.83', '安値: 42606.73', '終値: 42649.26']
['2025-08-14', '始値: 43129.18', '高値: 43199.83', '安値: 42606.73', '終値: 42649.26']
['2025-08-14', '始値: 43129.18', '高値: 43199.83', '安値: 42606.73', '終値: 42649.26']
['2025-08-14', '始値: 43129.18', '高値: 43199.83', '安値: 42606.73', '終値: 42649.26']
['2025-08-13', '始値: 43090.91', '高値: 43451.46', '安値: 43008.2', '終値: 43274.67']
['2025-08-13', '始値: 43090.91', '高値: 43451.46', '安値: 43008.2', '終値: 43274.67']
['2025-08-13', '始値: 43090.91', '高値: 43451.46', '安値: 43008.2', '終値: 43274.67']
['2025-08-13', '始値: 43090.91', '高値: 43451.